In [2]:
import pennylane as qml
import pennylane.numpy as np 

from sklearn.datasets import load_iris

from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [45]:
X, y = load_iris(return_X_y=True)

In [47]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [48]:
X = X[:100, :]
y = y[:100]

In [50]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=123)

((30, 4), (70, 4))

In [57]:
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
X_test.shape, X_train.shape

((30, 4), (70, 4))

kernel 
$$k(x_j, x_k)  = |\bra{0} \Phi^{\dag} (x_j) \Phi (x_k) \ket{0}|^2$$

In [59]:
num_qubits = 4 # bo 4 zmienne

dev = qml.device('default.qubit', wires=num_qubits)


In [40]:
@qml.qnode(dev)
def qc(a,b):
    qml.AngleEmbedding(features=a, wires=range(num_qubits), rotation='X')
    qml.adjoint(qml.AngleEmbedding(features=b, wires=range(num_qubits), rotation='X'))
    return qml.probs(wires=range(num_qubits))

In [64]:
X_train[0]

array([0.73913043, 0.84090909, 0.29411765, 0.25      ])

In [70]:
qc(X_train[0], X_train[0])

tensor([1.00000000e+00, 4.40652173e-38, 5.70726749e-35, 2.00826258e-36,
        1.74203515e-33, 5.00317179e-38, 2.19269390e-34, 2.28018454e-36,
        0.00000000e+00, 1.85855637e-39, 1.23130106e-38, 8.47032978e-38,
        2.10982474e-33, 9.61730693e-40, 1.56979860e-34, 4.38306648e-38], requires_grad=True)

In [41]:
def qkernel(A,B):
    return np.array([[qc(a,b)[0] for a in A] for b in B ])

In [60]:
qkernel[X_train,y_train]

TypeError: 'function' object is not subscriptable

In [42]:
qsvm = SVC(kernel=qkernel)
qsvm.fit(X_train, y_train)

SVC(kernel=<function qkernel at 0x17b112480>)

In [69]:
y_pred = qsvm.score(X_test, y_test)

ValueError: X.shape[1] = 30 should be equal to 70, the number of samples at training time

In [31]:
accuracy_score(y_pred=y_pred, y_true=y_train)

1.0

In [32]:
y_pred

array([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1])

In [33]:
y_train

array([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1])